# Metaprogramming

I linguaggi dinamici mettono a disposizione meccanismi di gestione avanzata del codice. Essi permettono di adattare a runtime il comportamente del programma senza aver bisogno di ricompilazione. Per permettere questo adattamento a situazione differenti è necessario che il linguaggio supporti alcune funzionalità:

- Capacità di auto analisi del codice a tempo di esecuzione
- Generazione e modifica del codice a tempo di esecuzione
- Intercettazione e gestione degli errori a tempo di esecuzione

## Introspection

L'introspection, introspezione, è la capacità di un linguaggio di determinare tipo e proprietà di un oggetto a tempo di esecuzione. Si presuppone quindi il type checking dinamico.

L'introspezione è utilizzata principalmente per l'ispezione di classi, attributi e metodi senza il bisogno di conoscere il nome a tempo di compilazione. L'introspezione è una proprietà tipica di linguaggi dinamici, ma esiste anche in altri non dinamici, come ad esempio Java.

Un esempio di introspezione è l'auto completazione del testo di un classico IDE. Infatti l'auto completamento avviene solamente a tempo di esecuzione del nostro IDE e non può conoscere a priori quello che noi andremo a scrivere. L'auto completamento procede prima con il recupero dei metodi pubblici dell'oggetto, controlla se sono callable ed infine ne presenta i nomi a noi sviluppatori.

### Duck typing

Il duck typing è un argomento che affronteremo meglio in un notebook sulla tipizzazione.

In ogni caso, proviamo comunque a spiegare brevemente cosa sia questo duck typing. Consideriamo un generico oggetto ```O``` e vogliamo utilizzare il suo metodo ```O.write()```. Il compilatore non esegue controlli sul fatto che questo metodo appartenga effettivamente all'oggetto, come invece farebbe un linguaggio come Java. L'eventuale errore si verificicherà quindi a tempo di esecuzione se il metodo non viene trovato.

Per controllare questo possibile errore abbiamo due modi. Il primo è il classico try except, mentre il secondo è l'utilizzo della funzione built-in ```hasattr(O, 'write')```, che è appunto uno strumento di introspezione.

### Primo esempio metaprogramming

Vediamo un primo esempio completo di metaprogramming. Immaginiamo di volere implementare in python il costrutto switch case. Vogliamo però renderlo dinamico, ovvero avere la possibilità di aggiungere e eliminare dei casi a runtime.

Vogliamo in sintesi sviluppare un costrutto così costruito:

```
case condizione of:
    caso_X: ...
    caso_Y: ...
    ...
    default: ...
```

La prima cosa che facciamo è creare una classe ```switch``` callable.

In [2]:
class Switch:
    def __call__(self, condizione):
        azione = getattr(self, f'caso_{condizione}', None)
        if not azione:
            return self.default
        return azione

Ora con la classe switch, creiamo una classe contenente tanti metodi quanti casi vogliamo implementare.

In [4]:
class TestConditions(Switch):
    def caso_1(self):
        return "Trattamento condizione 1"

    def caso_2(self):
        return "Trattamento condizione 2"

    def caso_A(self):
        return "Trattamento condizione A"

    def caso_BC(self):
        return "Trattamento condizione BC"

    def default(self):
        raise Exception("Condizione non prevista")

Dalla struttura del nostro codice, è abbastanza immediato capire quanto sia semplice implementare un caso direttamente a runtime. Basterebbe aggiungere un \__getattribute\__ e un \__getattr\__ per settare una nuova funzione come metodo di nostro switch. Risulta anche immediato come semplicemente indicando una stringa di caso d'uso, il nostro codice riesce a ricondursi al nostro metodo.

Ora ci basta solamente testare il nostro costrutto.

In [8]:
test = TestConditions()

print(test("1")())
print(test("BC")())
print(test("3")())

Trattamento condizione 1
Trattamento condizione BC


Exception: Condizione non prevista

### Strumenti primitivi per introspection

Python ci mette a disposizione vari strumenti per l'introspezione:

- ```type(obj)```

    Funzione built-in con un singolo parametro. Questa chiamata ritorna il tipo dell'oggetto obj

- ```isinstance(obj, cls)```

    Restituisce True se il tipo di obj è il tipo cls, altrimenti restituisce False

- ```obj.__dict__```

    È un attributo di ogni oggetto che memorizza un dizionario contenente tutti gli attributi con il relativo valore

- ```dir()``` e ```dir(obj)```

    La prima funzione built-in restituisce i nomi dell'ambiente corrente. La seconda funzione built-in, ovvero con un parametro, restituisce la lista più ampia possibile degli attributi dell'oggetto obj secondo il seguente algoritmo:

    - Se l'oggetto in questione obj implementa il metodo \__dict\__, allora viene utilizzaro questo metodo
    - Se l'oggetto obj è un modulo, restituisce i suoi attributi
    - Se l'oggetto obj è una classe, restituisce i suoi attributi e ricorsivamente quelli della classe base
    - Per ogni altro oggetto, restituisce i suoi attributi, quelli della sua classe e ricorsivamente quelli delle classi base

- ```locals()``` e ```globals(obj)```

    Queste due funzioni built-in li conosciamo già. Esse ci permettono di recuperare i relativi namespace

- ```vars()``` e ```vars(obj)```

    La prima funzione è analoga alla funzione locals(), mentre se la utilizziamo in aggiunta al parametro obj, essa ci ritorna il valore di __dict__ dell'oggetto obj se presente, altrimenti solleva un'eccezzione di tipo TypeError

- ```hasattr(obj, attr)```

    Questa funzione ha lo scopo di controllare se un dato oggetto obj contiene un certo attributo attr. In caso affermativo ritorna True, altrimenti False

- ```getattr(obj, attr)``` e ```getattr(obj, attr, default)```

    La funzione getattr ci permette di ritornare un determinato attributo attr dell'oggetto obj. Se questo attributo non risulta presente nell'oggetto obj, gettattr solleva un'eccezzione AttributeError nel caso non sia specificato il parametro default, altrimenti ritorna proprio esso, default

- ```callable(obj)```

    Questa funzione controlla se l'oggetto obj è di tipo callable, ovvero implementa un metodo \__call\__

- ```obj.__class__```

    L'attributo speciale \__class\__ ha lo scopo di memorizzare il tipo di classe dell'oggetto obj

- ```obj.__class__.__name__```

    L'attributo \__class\__.\__name\__, a differenza di \__class\__, memorizza solamente il nome della classe dell'oggetto obj

- ```id(obj)```

    La funzione built-in id() permette di ottenere l'identificatore univoco di un dato ooggetto. Ciò vuole dire in sostanza ottenre l'indirizzo fisico in memoria sul quale è salvato l'oggetto obj. Questo metodo ci permette di avere il massimo del controllo sugli oggetti

Iniziamo a vedere degli esempi in pratica di questi metodi di introspezione.

In [1]:
class A:
    x = 1

In [2]:
class B(A):
    'Classe di esempio'
    z = 0

    def __init__(self, y):
        self.y = y

Esempi di ```isinstance(obj, cls)```, ```obj.__dict__```, ```dir()``` e ```dir(obj)```.

In [5]:
a = A()
b = B(2)

print(f"a is instance of B? {isinstance(a, B)}")
print(f"b is instance of B? {isinstance(b, B)}")

print("b.__dict__:")
print(b.__dict__)

print("dir():")
print(dir())

print("dir(b):")
print(dir(b))

a is instance of B? False
b is instance of B? True
b.__dict__:
{'y': 2}
dir():
['A', 'B', 'In', 'Out', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_i5', '_ih', '_ii', '_iii', '_oh', '_rwho_ls', 'a', 'b', 'exit', 'get_ipython', 'os', 'quit', 'sys']
dir(b):
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'x', 'y', 'z']


Esempi di ```locals()```, ```globals()``` e ```vars()```.

In [6]:
# Attenzione: vars() lo utilizziamo senza parametri
print(f"locals() == vars()? {locals() == vars()}")

# Questa volta utilizziamo vars con un parametro, vars(b)
print("vars(b):")
print(vars(b))

locals() == vars()? True
vars(b):
{'y': 2}


Esempi di ```hasattr(obj, attr)``` e ```getattr(obj, attr)```.

In [13]:
print(f"Has b customName attribute? {hasattr(b, 'customName')}")
print(f"Has b z attribute? {hasattr(b, 'z')}")

print(f"Get attribute z from b: {getattr(b, 'z')}")
print(f"Get attribute customName from b: {getattr(b, 'customName', 'No attribute')}")

Has b customName attribute? False
Has b z attribute? True
Get attribute z from b: 0
Get attribute customName from b: No attribute


Esempi di ```callable(obj)```, ```obj.__class__``` e ```id(obj)```.

In [14]:
class C:
    def __call__():
        print("Called __call__")

In [25]:
c = C()

print(f"Is b callable? {callable(b)}")
print(f"Is c callable? {callable(c)}")

print("b.__class__:")
print(b.__class__)

print("b.__class.__name__:")
print(b.__class__.__name__)

l = [1, 2, 3]
m = [1, 2, 3]

print(f"Is l list equals [without id()] of m list? {l == m}")
print(f"Is l list equals [with id()] of m list? {id(l) == id(m)}")

Is b callable? False
Is c callable? True
b.__class__:
<class '__main__.B'>
b.__class.__name__:
B
Is l list equals [without id()] of m list? True
Is l list equals [with id()] of m list? False


## Reflection

La reflection differisce dalla introspection per una singola particolarità. Infatti, l'introspezione permette di analizzare il codice a tempo d esecizioine, mentre la riflessione permette non solo di auto analizzarsi, ma anche di modiificare il codice a runtime. È quindi un importante aspetto della meta programmazione.

Avevamo già accennato nei precedenti noteboook dei metodi che implementavano il concetto di reflection. Un esempio è proprio la creazione di classi a runtime grazie a type. Un'altra funzione molto utile per modificare il codice a tempo di esecuzione è l'analogo di getattr, ovvero ```setattr```.

La funzione built-in setattr, come possiamo immaginarci, permette di settare attributi, esistenti o non, di un qualsiasi oggetto oppure classe. Vediamo un esempio.

In [26]:
class A:
    x = 0

In [27]:
a = A()

print(f"a.x: {a.x}")

try:
    print(f"a.y: {a.y}")
except AttributeError:
    print("Attribute not found")

setattr(a, "y", 1)

print(f"a.y: {a.y}")

setattr(a, "x", 2)

print(f"a.x: {a.x}")

a.x: 0
Attribute not found
a.y: 1
a.x: 2


Con settattr è possibile aggiungere ad oggetti o classi non solamente attributi, ma anche vere e proprie funzioni.

In [31]:
def init(self):
    self.z = 3

setattr(A, "__init__", init)

b = A()

print(f"b.x: {b.x}")
try:
    print(f"b.y: {b.y}")
except AttributeError:
    print("Attribute not found")
print(f"b.z: {b.z}")

b.x: 0
Attribute not found
b.z: 3


Ogni oggetto, e quindi anche le funzioni essendo anch'esse first-class object, possiedono un attributo speciale, ```__code__```. Questo attributo permette di ottenere il codice di una data funzione. Vediamo cosa intendo.

In [2]:
def funzione():
    print("Ciao")

funzione()
print(funzione.__code__)

Ciao
<code object funzione at 0x1125ae0e0, file "<ipython-input-2-df91efa5c070>", line 1>


Questo attributo permette di rendere altamente dinamica la programmazione. Infatti questo ci permette di riassegnare il codice di una data funzione come codice di una seconda funzione, differente dalla prima.

In [6]:
def saluto():
    pass

saluto()    # Non esegue niente

saluto.__code__ = funzione.__code__

saluto()

Ciao


Abbiamo programmato una funzione dinamicamente, durante l'esecuzione del programma! Questo è meta programming.

Ma possiamo fare ancora qualcosa in più. Possiamo modificare il codice stesso ritornato dall'attributo \__code\__.

In [8]:
def one_time_only(message):
    one_time_only.__code__ = (lambda x: None).__code__
    print(message)

one_time_only("Ciao Mondo!")        # Esegue la stampa
one_time_only("Non stampa più!")    # Non esegue più la stampa

Ciao Mondo!


Abbiamo definito una funzione la quale al suo interno altera il proprio comportamento dopo la prima chiamata, molto interessante.

Ora analizziamo due funzioni built-in che applicano la reflection, ```eval()``` e ```exec```.

Queste due funzioni sono altamente utili, ma anche molto pericolose se non utilizzate correttamente. Queste due funzioini permettono di eseguire letteralmente del codice a runtime. Diamo un primo sguardo alle relative docstring.

In [10]:
help(eval)

help(exec)

Help on built-in function eval in module builtins:

eval(source, globals=None, locals=None, /)
    Evaluate the given source in the context of globals and locals.
    
    The source may be a string representing a Python expression
    or a code object as returned by compile().
    The globals must be a dictionary and locals can be any mapping,
    defaulting to the current globals and locals.
    If only globals is given, locals defaults to it.

Help on built-in function exec in module builtins:

exec(source, globals=None, locals=None, /)
    Execute the given source in the context of globals and locals.
    
    The source may be a string representing one or more Python statements
    or a code object as returned by compile().
    The globals must be a dictionary and locals can be any mapping,
    defaulting to the current globals and locals.
    If only globals is given, locals defaults to it.



Le documentazioni ci dicono che ricevono come parametri delle stringhe rappresentati un'espressione e un'insieme di statement, relativamente per eval e exec. Vediamo un primo esempio.

In [16]:
print(eval("2*3/4"))

exec("print(f'{2*3/4}')")

1.5
1.5


Una prima differenza che notiamo è proprio il valore di ritorno. La funzione eval esegue un'espressione, come fosse una sorta di calcolatrice. Ha quindi necessità di ritornare il valore che ha ricavato. La funzione exec invece esegue veri e propri statement, come fossero statement in un file python. Eseguendo di suo degli statement, exec non ha la necessità di ritornare un valore, ma al posto di esso ritorna, come side effect, il risultato del codice da esso eseguito.

Dalla documentazinoe ci viene detto che entrambe le funzioni possono ricevere come parametri dei namespace, ovvero i namespace globali e locali. Essi verranno passati sotto forma di dizionari.

In [17]:
print(eval("a*b/c", {'a': 2, 'b': 3, 'c': 4}))
print(eval("a*b/c", {'a': 2, 'b': 3, 'c': 4}, {'a': 5, 'b': 6, 'c': 7}))

exec("print(f'{a*b/c}')", {'a': 2, 'b': 3, 'c': 4})
exec("print(f'{a*b/c}')", {'a': 2, 'b': 3, 'c': 4}, {'a': 5, 'b': 6, 'c': 7})

1.5
4.285714285714286
1.5
4.285714285714286


Dai valori dei risultati riusciamo a comprendere come entrambe le funzioni utilizzano le regole a noi già conosciute, ovvero la regola LEGB. Infatti, nel caso il namespace locale viene definito, le funzioni utilizzeranno il primo namespace che trovano secondo la regola LEGB, quindi il namespace locale. Se invece esso non fosse definito, vengono utilizzate le variabili del namespace globale.

Per concludere, sempre la documentazione ci dice che le funzioni possono ricevere il codice anche sotto forma di oggetto compilato e non solamente come stringa. Questo è possibile grazie alla funzione ```compile()``` che permette di ritornare un oggetto creato appositamente per l'utilizzo di eval oppure exec.

In [25]:
code = '''print(f"{a*b/c}")'''

obj_eval = compile(code, "example", "eval")
obj_exec = compile(code, "example", "exec")

eval(obj_eval, {'a': 2, 'b': 3, 'c': 4})

exec(obj_exec, {'a': 2, 'b': 3, 'c': 4})

1.5
1.5


Questo esempio fa sorgere però qualche domanda. Come mai sia eval che exec eseguono ugualmente lo stesso codice? Eval non doveva eseguire solamente delle espressioni?

Proviamo a fare un pò di chiarezza. Eval esegue sia espressioni che statement singoli. Nel caso esegua espressioni, eval avrà un valore di ritorno pari al valore calcolato, mentre se esegue un singolo statement, come in questo esempio, eval non avrà un valore di ritorno ma avrà un side effect grazie allo statement. Exec, a differenza di eval, non può eseguire espressioni ma in aggiunta ha le capacità per eseguire non un singolo statement, ma intere porzioni di codice e quindi un insieme di vari statement.

Vediamo qualche esempio di questi casi.

In [30]:
# Exec non riesce ad eseguire espressioni
print(exec("2*3/4"))
# Ma solamente degli statement, del codice
print(exec("x=2*3/4"))
print(x)

None
None
1.5


In [34]:
# Eval non riesce ad eseguire più statement
code = '''print("Ciao")
print("A")
print("Tutti")'''

try:
    eval(code)
except SyntaxError:
    print("Non riesce")

Non riesce


## Gestione degli errori

Tutti i linguaggi dinamici necessitano di un meccanismo che permetta di intercettare e gestire gli errori che si presentano a tempo di esecuzione.

Il meccanismo oramai alla base della totalità dei linguaggi, non solo dinamici, è il meccanismo introdotto da Java. Java introdusse un meccanismo di gestione degli errori basato sulle eccezzioni software. La gestione degli errori è possibile suddividerla in due macro classi, un approccio basato su classi ed un'altro basato su istruzioni speciali.

Java, come Python e la maggior parte dei linguaggi, adottano la gestione degli errori basata su classi. Linguaggi come Perl invece adotta la gestione degli errori basata su istruzioni speciali, come eval e die, accennata nel relativo notebook su perl.

## Approccio basato su classi

Approfondiamo la gestione degli errori basata su classi. Il concetto è semplice, il runtime environment prevede una singola classe madre rappresentante una generica eccezzione. Essa dovrà contenere almeno le seguenti informazioni:

- Un identificativo dell'istanza eccezzione
- Una rappresentazione dello stack corrente

A questo punto, grazie alla classe madre, è possibile implementare una serie di eccezzioni solamente grazie al concetto di ereditarietà. Possiamo immaginarci una prima sottoclasse rappresentante le generiche eccezzioni riguardanti all'I/O e a sua volta contenere varie classi figlie le quali permettono di specificare ogni singola eccezione in maniera più dettagliata.

Le eccezzioni possono essere sollevate in due modi differenti:

- Automaticamente

    L'interprete a tempo di esecuzione incontra un'istruzione vietata, possiamo immaginarci una divisione per 0, quindi solleva un'eccezzione specificandone il problema a riguardo

- Manualmente

    Il programmatore stesso decide di sollevare un'eccezzione per motivi suoi. Questo avviene grazie ad istruzioni specifiche come ```throw``` e ```raise```

Generalmente, incontrare un'eccezzione permette di definire un codice specifico per gestire l'errore stesso. Nel caso questo codice di gestione dell'errore non fosse definito, allora viene eseguito il gestore di default e quindi termina l'esecuzione del programma stampando lo stack corrente.

Python, come detto prima, basa la gestione degli errori come in Java, ovvero su classi e con le istruzioni ```try``` - ```except``` - ```finally```, le quali corrispondono alle istruzioni java ```try``` - ```catch``` - ```finally```. Questa di seguito è una generica struttura che permette la gestione di una o più generiche eccezzioni.

```python
try:
    <Blocco di codice>
except Eccezzione e_1:
    <Gestione dell'eccezzione>
except Eccezzione e_n:
    <Gestione dell'eccezzione>
else:
    <Codice eseguito in assenza di qualsiasi precedente eccezzione>
finally:
    <Codice eseguito in ogni caso alla fine del try - except>
```

In python è possibile inoltre raggruppare un numero generico di eccezzioni dentro un unico blocco per la gestione di essi.

```python
try:
    <Blocco di codice>
except (Eccezzione e_1, ... Eccezzione e_n):
    <Gestione dell'eccezzione>
else:
    <Codice eseguito in assenza di qualsiasi precedente eccezzione>
finally:
    <Codice eseguito in ogni caso alla fine del try - except>
```

Vediamo un semplice esempio per dimostrare quanto detto.

In [35]:
while True:
    try:
        num = int(input("Inserisci un intero: "))
    except ValueError:
        print("Non hai inserito un intero!")
        print("Riprova...")
    else:
        print("Hai inserito un intero, bravo!")
        break

Non hai inserito un intero!
Riprova...
Non hai inserito un intero!
Riprova...
Hai inserito un intero, bravo!


La struttura degli errori basata su classi abbiamo capito che è altamente estendibile. Infatti essa ci permette addirittura di definire delle eccezzioni definite dal programmatore stesso. Essa dovrà quindi essere una sottoclasse della classe padre Exception ed essere definita come segue.

In [36]:
class MyError(Exception):
    def __init__(self, message="It's my error!", errors=None):
        super().__init__(message)
        self.errors = errors

A questo punto possiamo testare la nostra eccezzione appena definita con l'istruzione ```raise```. Essa infatti ci permette di sollevare un'eccezzione manualmente.

In [37]:
raise MyError

MyError: It's my error!

In [38]:
raise MyError("Another error message")

MyError: Another error message

Quando si scrive un codice per la gestione degli errori è possibile voler ottenere specifiche informazioni dall'errore stesso, banalmente per fare del debugging. Per fare ciò si può utilizzare l'istruzione ```as``` segeuita dal nome che vogliamo dare all'istanza dell'eccezzione in questione.

In [39]:
import traceback

try:
    raise MyError("Another another error message")
except MyError as instance:
    print(instance.args)
    print(instance.errors)
    traceback.print_exc()

('Another another error message',)
None
Traceback (most recent call last):
  File "<ipython-input-39-b88fbf193f56>", line 4, in <module>
    raise MyError("Another another error message")
MyError: Another another error message
